<a href="https://colab.research.google.com/github/HarryZheng27/pythonstuff/blob/main/AI_vision_pipeline_for_photonvision.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%cd /content
%pip install "ultralytics<=8.3.40"

#getting dataset from roboflow universe
%cd /content
!pip install roboflow

from roboflow import Roboflow
rf = Roboflow(api_key="AFpSrWmhOiSGorei8cOh")
project = rf.workspace("zerru").project("humanfinder-8wu97")
version = project.version(1)
dataset = version.download("yolov8")

#rename to "datasets"
import os

# Path to the directory containing the folders
parent_dir = "/content/"

# List all items in the directory
items = os.listdir(parent_dir)

# Filter for directories only, excluding "sample-data"
folders = [f for f in items if os.path.isdir(os.path.join(parent_dir, f)) and f != "sample_data" and f != ".config"]

if len(folders) != 1:
    raise RuntimeError(f"Expected exactly 1 folder besides 'sample-data', found {len(folders)}: {folders}")

# The folder to rename
old_name = folders[0]
old_path = os.path.join(parent_dir, old_name)
new_path = os.path.join(parent_dir, "datasets")

# Rename the folder
os.rename(old_path, new_path)

print(f"Renamed '{old_name}' → 'datasets'")

/content
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 898.5/898.5 kB 24.2 MB/s eta 0:00:00
/content
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 89.8/89.8 kB 6.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.8/66.8 kB 6.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.9/49.9 MB 21.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 87.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.2/4.2 MB 106.7 MB/s eta 0:00:00
  Attempting uninstall: opencv-python-headless
    Found existing installation: opencv-python-headless 4.12.0.88
    Uninstalling opencv-python-headless-4.12.0.88:
      Successfully uninstalled opencv-python-headless-4.12.0.88
  Attempting uninstall: idna
    Found existing installation: idna 3.10
    Uninstalling idna-3.10:
      Successfully uninstalled idna-3.10
loading Roboflow workspace...
loading Roboflow project...



Extracting Dataset Version Zip to HumanFinder-1 in yolov8:: 100%|██████████| 20870/20870 [00:02<00:00, 7069.17it/s] 


Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.
Renamed 'HumanFinder-1' → 'datasets'


In [ ]:
#training
%cd /content/datasets
!yolo train model=yolov8s.pt data=data.yaml epochs=5 imgsz=640

#convert to onnx
%pip install numpy
!yolo export model=/content/datasets/runs/detect/train/weights/best.pt format=onnx

/content/datasets
100% 21.5M/21.5M [00:00<00:00, 131MB/s]
New https://pypi.org/project/ultralytics/8.3.208 available 😃 Update with 'pip install -U ultralytics'
Ultralytics 8.3.40 🚀 Python-3.12.11 torch-2.8.0+cu126 CUDA:0 (Tesla T4, 15095MiB)
engine/trainer: task=detect, mode=train, model=yolov8s.pt, data=data.yaml, epochs=5, time=None, patience=100, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=train, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=Fa

In [ ]:
%cd /content
dataset_txt = "dataset.txt"

print("Creating dataset.txt for quantization...")
with open(dataset_txt, "w") as f:
    img_dir = os.path.join('datasets', "train", "images")
    for img_file in os.listdir(img_dir):
        if img_file.lower().endswith((".jpg", ".png", ".jpeg")):
            f.write(os.path.join(img_dir, img_file) + "\n")

/content
Creating dataset.txt for quantization...


In [ ]:

#download rknntoolkit dependencies and itself
#make sure session is restarted to apply updates.
%cd /content
!git clone https://github.com/airockchip/rknn-toolkit2.git

!python3 --version
%cd /content/rknn-toolkit2/
!pip install -r rknn-toolkit2/packages/x86_64/requirements_cp312-2.3.2.txt
!pip install rknn-toolkit2/packages/x86_64/rknn_toolkit2-2.3.2-cp312-cp312-manylinux_2_17_x86_64.manylinux2014_x86_64.whl



/content
fatal: destination path 'rknn-toolkit2' already exists and is not an empty directory.
Python 3.12.11
/content/rknn-toolkit2
Processing ./rknn-toolkit2/packages/x86_64/rknn_toolkit2-2.3.2-cp312-cp312-manylinux_2_17_x86_64.manylinux2014_x86_64.whl


In [ ]:
!pip install onnx==1.18.0 --force-reinstall

  Using cached onnx-1.18.0-cp312-cp312-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (6.9 kB)
  Using cached numpy-2.3.3-cp312-cp312-manylinux_2_27_x86_64.manylinux_2_28_x86_64.whl.metadata (62 kB)
  Using cached protobuf-6.32.1-cp39-abi3-manylinux2014_x86_64.whl.metadata (593 bytes)
  Using cached typing_extensions-4.15.0-py3-none-any.whl.metadata (3.3 kB)
Using cached onnx-1.18.0-cp312-cp312-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (17.6 MB)
Using cached numpy-2.3.3-cp312-cp312-manylinux_2_27_x86_64.manylinux_2_28_x86_64.whl (16.6 MB)
Using cached protobuf-6.32.1-cp39-abi3-manylinux2014_x86_64.whl (322 kB)
Using cached typing_extensions-4.15.0-py3-none-any.whl (44 kB)
  Attempting uninstall: typing_extensions
    Found existing installation: typing_extensions 4.15.0
    Uninstalling typing_extensions-4.15.0:
      Successfully uninstalled typing_extensions-4.15.0
  Attempting uninstall: protobuf
    Found existing installation: protobuf 4.25.4
    Uninstalling protobu

In [ ]:
from rknn.api import RKNN
rknn = RKNN(verbose=True) # verbose=True for detailed output
rknn.config(mean_values=[[0, 0, 0]], std_values=[[255, 255, 255]], target_platform='rk3588')
ONNX_MODEL_PATH = '/content/best.onnx'
rknn.load_onnx(model=ONNX_MODEL_PATH)
rknn.build(do_quantization=True, dataset = "/content/dataset.txt")# do_quantization=True for quantization, dataset for calibration
RKNN_MODEL_PATH = '/content/model.rknn'
rknn.export_rknn(RKNN_MODEL_PATH)
rknn.release()

I rknn-toolkit2 version: 2.3.2
I Loading : 100%|███████████████████████████████████████████████| 144/144 [00:00<00:00, 8577.92it/s]
D base_optimize ...
/tmp/ipython-input-486623951.py:6: DeprecationWarning: label() is deprecated. Use is_required() or is_repeated() instead.
  rknn.build(do_quantization=True, dataset = "/content/dataset.txt")# do_quantization=True for quantization, dataset for calibration
D base_optimize done.
D 
D fold_constant ...
D fold_constant done.
D 
D correct_ops ...
D correct_ops done.
D 
D fuse_ops ...
D fuse_ops results:
D     replace_exswish: remove node = ['/model.0/act/Sigmoid', '/model.0/act/Mul'], add node = ['/model.0/act/Sigmoid_2swish']
D     replace_exswish: remove node = ['/model.1/act/Sigmoid', '/model.1/act/Mul'], add node = ['/model.1/act/Sigmoid_2swish']
D     replace_exswish: remove node = ['/model.2/cv1/act/Sigmoid', '/model.2/cv1/act/Mul'], add node = ['/model.2/cv1/act/Sigmoid_2swish']
D     replace_exswish: remove node = ['/model.2/m.0/cv1/a